# 自动写诗

## 定义 Config

In [2]:
# Prepare
import torch
import visdom
import tqdm
import ipdb
import numpy as np
import torchvision as tv

from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from torchnet import meter
from torch.autograd import Variable


# Config

class Config(object):

    lr = 1e-3

    epoch = 20
    batch_size = 128
    
    embedding_dim = 128
    hidden_dim = 256

    max_gen_len = 200

opt = Config()


## 定义 Visualizer 可视化

In [3]:
import visdom
import torch as t
import time
import torchvision as tv
import numpy as np


class Visualizer():
    """
    封装了visdom的基本操作，但是你仍然可以通过`self.vis.function`
    调用原生的visdom接口
    """

    def __init__(self, env='default', **kwargs):
        import visdom
        self.vis = visdom.Visdom(env=env, use_incoming_socket=False, **kwargs)

        # 画的第几个数，相当于横座标
        # 保存（’loss',23） 即loss的第23个点
        self.index = {}
        self.log_text = ''

    def reinit(self, env='default', **kwargs):
        """
        修改visdom的配置
        """
        self.vis = visdom.Visdom(env=env,use_incoming_socket=False, **kwargs)
        return self

    def plot_many(self, d):
        """
        一次plot多个
        @params d: dict (name,value) i.e. ('loss',0.11)
        """
        for k, v in d.items():
            self.plot(k, v)

    def img_many(self, d):
        for k, v in d.items():
            self.img(k, v)

    def plot(self, name, y):
        """
        self.plot('loss',1.00)
        """
        x = self.index.get(name, 0)
        self.vis.line(Y=np.array([y]), X=np.array([x]),
                      win=name,
                      opts=dict(title=name),
                      update=None if x == 0 else 'append'
                      )
        self.index[name] = x + 1

    def img(self, name, img_):
        """
        self.img('input_img',t.Tensor(64,64))
        """

        if len(img_.size()) < 3:
            img_ = img_.cpu().unsqueeze(0)
        self.vis.image(img_.cpu(),
                       win=name,
                       opts=dict(title=name)
                       )

    def img_grid_many(self, d):
        for k, v in d.items():
            self.img_grid(k, v)

    def img_grid(self, name, input_3d):
        """
        一个batch的图片转成一个网格图，i.e. input（36，64，64）
        会变成 6*6 的网格图，每个格子大小64*64
        """
        self.img(name, tv.utils.make_grid(
            input_3d.cpu()[0].unsqueeze(1).clamp(max=1, min=0)))

    def log(self, info, win='log_text'):
        """
        self.log({'loss':1,'lr':0.0001})
        """

        self.log_text += ('[{time}] {info} <br>'.format(
            time=time.strftime('%m%d_%H%M%S'),
            info=info))
        self.vis.text(self.log_text, win=win)

    def __getattr__(self, name):
        return getattr(self.vis, name)

## 定义模型 PoetryModel

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PoetryModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(PoetryModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_dim, num_layers=2)
        self.linear = nn.Linear(self.hidden_dim, vocab_size)

    def forward(self, input, hidden=None):
        seq_len, batch_size = input.size()
        if hidden is None:
            h_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            c_0 = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
        else:
            h_0, c_0 = hidden
            
        embeds = self.embeddings(input)

        output, hidden = self.lstm(embeds, (h_0, c_0))

        output = self.linear(output.view(seq_len * batch_size, -1))
        
        return output, hidden

## 数据准备

In [5]:
def prepareData():
    datas = np.load("data/tang.npz", allow_pickle=True)
    print(datas)
    data = datas['data']
    ix2word = datas['ix2word'].item()
    word2ix = datas['word2ix'].item()
    data = torch.from_numpy(data)
    dataloader = DataLoader(data,
                            batch_size=opt.batch_size,
                            shuffle=True,
                            num_workers=2)
    return dataloader, ix2word, word2ix

### 测试

In [6]:
dataloader, ix2word, _ = prepareData()
for batch_size, data in enumerate(dataloader):
    poem1 = data[0]
    poem = ""
    for i in poem1:
        poem += ix2word[int(i)]
    print(poem)
    break

</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s><START>紫髯青眼代天才，韩白孙吴稍可陪。祗见赤心尧日下，岂知真气梵天来。听经瑞雪时时落，登塔天花步步开。尽祝庄椿同壽考，人间岁月岂能催。<EOP>


## 生成诗句

In [12]:
def generate(model, start_words, ix2word, word2ix, prefix_words = None):
    results = list(start_words)
    start_words_len = len(start_words)

    # 第一个词语是<START>
    input = t.Tensor([word2ix['<START>']]).view(1, 1).long().cuda()
    hidden = None

    if prefix_words != None:
        for word in prefix_words:
            output, hidden = model(input, hidden)
            input = input.data.new([word2ix[word]]).view(1, 1)

    for i in range(opt.max_gen_len):
        output, hidden = model(input, hidden)
        # 如果在给定的句首中，input为句首中的下一个字
        if i < start_words_len:
            w = results[i]
            input = input.data.new([word2ix[w]]).view(1, 1)
        # 否则将output作为下一个input进行
        else:
            top_index = output.data[0].topk(1)[1][0].item()
            w = ix2word[top_index]
            results.append(w)
            input = input.data.new([top_index]).view(1, 1)
        if w == '<EOP>':
            del results[-1]
            break
    return results

# def generate(model, start_words, ix2word, word2ix):
#     results = list(start_words)
#     start_words_len = len(start_words)
#     # 第一个词语是<START>
#     input = t.Tensor([word2ix['<START>']]).view(1, 1).long().cuda()
#     hidden = None
#     model.eval().cuda()
#     with torch.no_grad():
#         for i in range(Config.max_gen_len):
#             output, hidden = model(input, hidden)
# 		    # 如果在给定的句首中，input为句首中的下一个字
#             if i < start_words_len:
#                 w = results[i]
#                 input = input.data.new([word2ix[w]]).view(1, 1)
#            # 否则将output作为下一个input进行
#             else:
#                 top_index = output.data[0].topk(1)[1][0].item()
#                 w = ix2word[top_index]
#                 results.append(w)
#                 input = input.data.new([top_index]).view(1, 1)
#             if w == '<EOP>':
#                 del results[-1]
#                 break
#         return results


def gen_acrostic(model, start_words, ix2word, word2ix, prefix_words=None):

    results = []
    start_word_len = len(start_words)
    input = (t.Tensor([word2ix['<START>']]).view(1, 1).long()).cuda()
    hidden = None

    index = 0  # 用来指示已经生成了多少句藏头诗
    # 上一个词
    pre_word = '<START>'

    if prefix_words:
        for word in prefix_words:
            output, hidden = model(input, hidden)
            input = (input.data.new([word2ix[word]])).view(1, 1)

    for i in range(opt.max_gen_len):
        output, hidden = model(input, hidden)
        top_index = output.data[0].topk(1)[1][0].item()
        w = ix2word[top_index]

        if (pre_word in {u'。', u'！', '<START>'}):
            # 如果遇到句号，藏头的词送进去生成

            if index == start_word_len:
                # 如果生成的诗歌已经包含全部藏头的词，则结束
                break
            else:
                # 把藏头的词作为输入送入模型
                w = start_words[index]
                index += 1
                input = (input.data.new([word2ix[w]])).view(1, 1)
        else:
            # 否则的话，把上一次预测是词作为下一个词输入
            input = (input.data.new([word2ix[w]])).view(1, 1)
        results.append(w)
        pre_word = w
    return results

## 训练模型

In [8]:
def train(dataloader, word2ix):
    vis = Visualizer(env="autoPoem")
    # 定义模型
    model = PoetryModel(len(word2ix),
                      embedding_dim=opt.embedding_dim,
                      hidden_dim=opt.hidden_dim)

    optimizer = optim.Adam(model.parameters(), lr=opt.lr)

    criterion = nn.CrossEntropyLoss()
    model.cuda()
    criterion.cuda()

    loss_meter = meter.AverageValueMeter()

    for epoch in range(opt.epoch):
        loss_meter.reset()

        for i, data in tqdm.tqdm(enumerate(dataloader)):
            model.train()
            data = data.long().transpose(1,0).contiguous().cuda()

            optimizer.zero_grad()

            input, target = data[:-1,:], Variable(data[1:,:])

            output,_ = model(input)
            loss = criterion(output, target.view(-1))
            loss.backward()
            optimizer.step()

            loss_meter.add(loss.item())

            if (1+i) % 100 == 0:
                vis.plot('loss', loss_meter.value()[0])
                poetrys = [[ix2word[_word] for _word in data[:, _iii].tolist()]
                           for _iii in range(data.shape[1])][:16]

                vis.text('</br>'.join([''.join(poetry) for poetry in poetrys]),win = u'origin_poem')

                gen_poetries = []

                for word in list(u'深度学习真的有趣'):
                    gen_poetry = ''.join(generate(model, word, ix2word, word2ix))
                    gen_poetries.append(gen_poetry)
                vis.text('</br>'.join([''.join(poetry) for poetry in gen_poetries]), win=u'gen_poem')

        t.save(model.state_dict(), 'model/pytorch/model_epoch_%s.pth' %epoch)

## 开始训练

In [9]:
dataloader, ix2word, word2ix = prepareData()
train(dataloader, word2ix)

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.
450it [00:27, 16.23it/s]
450it [00:27, 16.32it/s]
450it [00:27, 16.28it/s]
450it [00:27, 16.11it/s]
450it [00:28, 16.06it/s]
450it [00:28, 15.62it/s]
450it [00:28, 15.52it/s]
450it [00:28, 15.55it/s]
450it [00:28, 15.75it/s]
450it [00:28, 15.52it/s]
450it [00:29, 15.28it/s]
450it [00:29, 15.15it/s]
450it [00:30, 14.72it/s]
450it [00:31, 14.34it/s]
450it [00:30, 14.63it/s]
450it [00:30, 14.69it/s]
450it [00:30, 14.67it/s]
450it [00:30, 14.74it/s]
450it [00:30, 14.59it/s]
450it [00:31, 14.51it/s]


## 生成诗歌

In [18]:
# Load model 
dataloader, ix2word, word2ix = prepareData()
model = PoetryModel(len(word2ix),
                      embedding_dim=opt.embedding_dim,
                      hidden_dim=opt.hidden_dim)

map_location = lambda s, l: s
state_dict = torch.load('model/pytorch/model_epoch_19.pth', map_location=map_location)

model.load_state_dict(state_dict)
model.cuda()

start_words = u'枯藤老树昏鸦'
prefix_words = u'枯藤老树昏鸦小桥流水人家'


result = generate(model, start_words, ix2word, word2ix, prefix_words)
print(''.join(result))

枯藤老树昏鸦，一片红莲叶下枝。一枝一朵红叶绿，一朵红霞红粉垂。一朵红霞映碧落，一朵红莲映春色。一枝红豔豔红妆，一朵红妆红粉鲜。红粉红妆不可见，红妆粉黛如相看。不知不觉春风起，不觉春风吹落花。不觉春风吹不得，不知何处是春时。不知此日无人识，不觉春风不可知。自有春风吹不得，不知何处是何时？花间不得无人见，花落春来不可知。自有春风吹不得，不知何处是何时？花间不见无人识，花落春来不可知。自有春风吹不得，不


![avatar](data/1.svg)

![avatar](data/2.png)